In [11]:
import pandas as pd
import numpy as np

import glob
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import euclidean_distances#transform dataframe into rdm

import matplotlib.pyplot as plt
import seaborn as sns

# turn model result into RDM

In [12]:
def cluster_rdm(cluster_results_159r_within,cluster_model='cl_k6'):
    dissim_cluster_result = pd.DataFrame(index = cluster_results_159r_within.index,
                                         columns = cluster_results_159r_within.index)
    for i in dissim_cluster_result.index:
        for j in dissim_cluster_result.columns:
            # cluster result was in the last column
            if cluster_results_159r_within.loc[i,cluster_model] == cluster_results_159r_within.loc[j,cluster_model]:
                dissim_cluster_result.loc[i,j] = 0
            else:
                dissim_cluster_result.loc[i,j] = 1
            dissim_cluster_result = dissim_cluster_result.astype(float)
            
    return(dissim_cluster_result)

In [13]:
exp_cluster_within_data_paths = glob.glob('../output_data/individual/category/cluster_results/exp/*.csv')
imp_cluster_within_data_paths = glob.glob('../output_data/individual/category/cluster_results/imp/*.csv')

dim_3cluster_within_data_paths = glob.glob('../output_data/individual/dimension/cluster_results/pca_3k/*.csv')
dim_umap_cluster_within_data_paths = glob.glob('../output_data/individual/dimension/cluster_results/umap/*.csv')

## dimension survey

In [14]:
# pca data_3k
for ind_dir in dim_3cluster_within_data_paths:
    ind_input = pd.read_csv(ind_dir, index_col=0)
    ind_cluster_rdm = cluster_rdm(ind_input,cluster_model='cl_k3')
    
    ind_csv = ind_dir.split('\\')[1]
    ind_cluster_rdm.to_csv('../output_data/individual/dimension/cluster_RDMs/pca_3k/'+ind_csv)

In [15]:
# umap data
for ind_dir in dim_umap_cluster_within_data_paths:
    ind_input = pd.read_csv(ind_dir, index_col=0)
    ind_cluster_rdm = cluster_rdm(ind_input,cluster_model='cl_k3')
    
    ind_csv = ind_dir.split('\\')[1]
    ind_cluster_rdm.to_csv('../output_data/individual/dimension/cluster_RDMs/umap/'+ind_csv)

## category task

In [16]:
# explicit task
for ind_dir in exp_cluster_within_data_paths:
    ind_input = pd.read_csv(ind_dir, index_col=0)
    ind_cluster_rdm = cluster_rdm(ind_input,cluster_model='cl_k6')
    
    ind_csv = ind_dir.split('\\')[1]
    ind_cluster_rdm.to_csv('../output_data/individual/category/cluster_RDMs/exp/'+ind_csv)

In [17]:
# implicit task
for ind_dir in imp_cluster_within_data_paths:
    ind_input = pd.read_csv(ind_dir, index_col=0)
    ind_cluster_rdm = cluster_rdm(ind_input,cluster_model='cl_k3')
    
    ind_csv = ind_dir.split('\\')[1]
    ind_cluster_rdm.to_csv('../output_data/individual/category/cluster_RDMs/imp/'+ind_csv)

# LOOCV

## import data

In [18]:
dim_pca_3k_within_data_paths = glob.glob('../output_data/individual/dimension/cluster_RDMs/pca_3k/*.csv') 
dim_umap_within_data_paths = glob.glob('../output_data/individual/dimension/cluster_RDMs/umap/*.csv')

ca_within_data_paths = glob.glob('../output_data/individual/category/cluster_RDMs/exp/*.csv') 
ma_within_data_paths = glob.glob('../output_data/individual/category/cluster_RDMs/imp/*.csv') 

all were used 'spearman'

In [19]:
def LOOCV(subj_data_paths):
    rsa_list = []
    sub_id_list = []
    for left_one in subj_data_paths:
        sub_id_list.append(left_one.split('\\')[1].split('.')[0])
        # get the one left
        left_one_df = pd.read_csv(left_one, index_col=0)
        dissim_left_one = left_one_df
        
        # remove one subject from all 14 subjects
        subj_data_paths_remove_one = subj_data_paths.copy()
        subj_data_paths_remove_one.remove(left_one)
        
        dissim_remove_one = np.zeros((159,159))

        for f in subj_data_paths_remove_one:
            temp_df = pd.read_csv(f, index_col=0)
            dissim_remove_one += np.array(temp_df)

        dissim_remove_one_avg = dissim_remove_one / len(subj_data_paths_remove_one)
        dissim_remove_one_avg = pd.DataFrame(dissim_remove_one_avg, index=temp_df.index, columns=temp_df.columns)
        
        # caculate the correlation between the one left and others
        array_dissim_left_one = np.array(dissim_left_one)[np.triu_indices(159, k = 1)] 
        array_dissim_remove_one_avg = np.array(dissim_remove_one_avg)[np.triu_indices(159, k = 1)]
           
        rsa_list.append(spearmanr(array_dissim_remove_one_avg, array_dissim_left_one)[0])
        
    print(len(subj_data_paths_remove_one))
    
    rsa_list_df = pd.DataFrame({'sub_id':sub_id_list,'rsa':rsa_list})
    return rsa_list_df

In [20]:
exp_cluster_loocv = LOOCV(subj_data_paths = ca_within_data_paths)
imp_cluster_loocv = LOOCV(subj_data_paths = ma_within_data_paths)
dim_pca_3k_cluster_loocv = LOOCV(subj_data_paths = dim_pca_3k_within_data_paths)
dim_umap_cluster_loocv = LOOCV(subj_data_paths = dim_umap_within_data_paths)

13
13
13
13
